In [1]:
import csv
import numpy as np
import pandas as pd
import random

from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

## Data Processing

In [2]:
train = pd.read_csv('data.csv', sep=",")
test_data = pd.read_csv('quiz.csv', sep=",")

train_data = train.iloc[:,:-1]
train_labels = train.iloc[:,-1]

train_data, holdout_data, train_labels, holdout_label = train_test_split(train_data, train_labels, 
                                                                         test_size=0.1, random_state=1)
all_data = pd.concat([train_data, test_data])
train_obs = len(train_data)
test_obs = len(test_data)

# Change dtype of categorical columns
categorical_columns = ['0','5','7','8','9','14','16','17','18','20','23','25','26','56','57','58']
for i in range(0,len(categorical_columns)):
    all_data[categorical_columns[i]] = all_data[categorical_columns[i]].astype('category')
    
# Only numerical data
print('Processing numerical data...')
all_data_num = all_data.drop(categorical_columns, axis=1)
train_data_num = all_data_num.iloc[0:train_obs,]
test_data_num = all_data_num.iloc[train_obs:,]

# Only categorical data
print('Processing categorical data...')
all_data_cat = pd.get_dummies(all_data[categorical_columns])
train_data_cat = all_data_cat.iloc[0:train_obs,]
test_data_cat = all_data_cat.iloc[train_obs:,]

# Combine the two
print('Combining data...')
train_data_combo = pd.concat([train_data_num, train_data_cat], axis=1)
test_data_combo = pd.concat([test_data_num, test_data_cat], axis=1)

# Clear memory
all_data, train, train_data, test_data = None, None, None, None
all_data_num, train_data_num, test_data_num = None, None, None
all_data_cat, train_data_cat, test_data_cat = None, None, None

print('Processing finished!')

Processing numerical data...
Processing categorical data...
Combining data...
Processing finished!


## Parameter Tuning

In [4]:
params = {
    'n_estimators': randint(10, 100),
    'max_features': uniform(0.25,1),
    'random_state': [1],
    'n_jobs': [-1]
}
classifier = BaggingClassifier()
search_results = RandomizedSearchCV(estimator = classifier,
                                        param_distributions = params,
                                        n_iter = 3, n_jobs = 1,
                                        cv = 3, verbose = 3 )
# GridLogiClassifier = GridSearchCV(estimator = classifier, param_grid=params, n_jobs=1, cv=4, verbose=2)
search_results.fit(train_data_combo, train_labels)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] max_features=0.250989209792, n_estimators=94, n_jobs=-1, random_state=1 


/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

KeyboardInterrupt: 

In [ ]:
search_results.best_score_

In [ ]:
search_results.grid_scores_